-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

# 02L - Embeddings, Vector Databases, and Search


In this lab, we will apply the text vectorization, search, and question answering workflow that you learned in the demo. The dataset we will use this time will be on talk titles and sessions from [Data + AI Summit 2023](https://www.databricks.com/dataaisummit/). 

### ![Dolly](https://files.training.databricks.com/images/llm/dolly_small.png) Learning Objectives
1. Learn how to use Chroma to store your embedding vectors and conduct similarity search
1. Use OpenAI GPT-3.5 to generate response to your prompt

In [0]:
%pip install chromadb==0.3.21 tiktoken==0.3.3

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 1.3 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 966.2/966.2 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 35.4 MB/s eta 0:00:00
  Using cached numpy-1.25.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.2 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.4 MB/s 

## Classroom Setup

In [0]:
%run ../Includes/Classroom-Setup

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


Resetting the learning environment:
| enumerating serving endpoints...found 0...(0 seconds)
| No action taken

Skipping install of existing datasets to "dbfs:/mnt/dbacademy-datasets/large-language-models/v01"

Validating the locally installed datasets:
| listing local files...(9 seconds)
| validation completed...(9 seconds total)


Importing lab testing framework.



Using the "default" schema.

Predefined paths variables:
| DA.paths.working_dir: /dbfs/mnt/dbacademy-users/labuser4407984@vocareum.com/large-language-models
| DA.paths.user_db:     dbfs:/mnt/dbacademy-users/labuser4407984@vocareum.com/large-language-models/database.db
| DA.paths.datasets:    /dbfs/mnt/dbacademy-datasets/large-language-models/v01

Setup completed (24 seconds)

The models developed or used in this course are for demonstration and learning purposes only.
Models may occasionally output offensive, inaccurate, biased information, or harmful instructions.


## Read data

In [0]:
import pandas as pd

dais_pdf = pd.read_parquet(f"{DA.paths.datasets}/dais/dais23_talks.parquet")
display(dais_pdf)

Title,Abstract
Nebula: The Journey of Scaling Instacart’s Data Pipelines with Apache Spark™ and Lakehouse,"Instacart has gone through immense growth during the pandemic and the trend continues. Instacart ads is no exception in this growth story. We have launched many new product lines including display and video ads covering the full advertising funnel to address the increasing demand of our retail partners. We have built advanced models to auto-suggest optimal bidding to increase the ROI for our CPG partners. Advertisers’ trust is the utmost priority and thus the quest to build a top-class ads measurement platform. Ads data processing requires complex data verifications to update ads serving stats. In ETL pipelines these were implemented through files containing thousands of lines of raw SQL which were hard to scale, test, and iterate upon. Our data engineers used to spend hours testing small changes due to a lack of local testing mechanisms. These pain points stress our need for better tools. After some research, we chose Apache Spark™ as our preferred tool to rebuild ETLs, and the Databricks platform made this move easier. In this presentation, I will share our journey to move our pipelines to Spark and Delta Lake on Databricks. With spark, scala, and delta we solved many problems which were slowing the team’s productivity. Some key areas I will cover include: - Modular and composable code - Unit testing framework - Incremental event processing with spark structured streaming - Granular resource tuning for better performance and cost efficacy Other than the domain business logic, the problems discussed here are quite common for performing data processing at scale. I would be glad to have the opportunity to share our learning and am hopeful that it will benefit others who are going through similar growth challenges or migrating to Lakehouse."
Satellite Imaginary Data Processing Using Apache Spark™ and H3 Geospatial Indexing System,"Agriculture is a complex ecosystem. Understanding Ag data around soil metrics, weather and historical crop production is a key to adopt sustainable practices which help farmers to enhance their profitability and soil health. As these datasets are huge and disparate in nature, finding a standard unit of analysis and bringing all the data to a common granularity is challenging. By leveraging the distributed data processing capabilities of Apache Spark™ and h3 geospatial indexing system, created a hexagonal grid and mapped all the data sets using h3 index. This gave us an ability to join all the datasets together and helped us in deriving more insights. This session will share our learnings and experiences with the Spark community."
From Snowflake to Enterprise-Scale Apache Spark™,"Akamai mPulse is a real user monitoring (RUM) solution that delivers real-time web performance analytics to Akamai customers through dashboards, alerting, reporting and data science. The architecture of mPulse relies on a combination of public and private cloud-based services, such as Amazon AWS, Microsoft Azure and the Snowflake data warehouse. Snowflake has provided the core data warehousing needs as the product has grown at scale along with Akamai’s customers. The engineering team at mPulse has been re-architecting the system to migrate away from Snowflake to an internal enterprise-scale Apache Spark™ solution that Akamai has been developing in-house to improve performance and save on cost. In the first half of the talk, we’ll discuss how the mPulse team made the decision to migrate, the challenges we’ve seen and how Spark is suiting the product's needs. In the second half of the talk, we’ll discuss the details of the Spark-based infrastructure. Akamai data warehouse (a.k.a Asgard) is a Spark-based solution running on the Azure cloud. We will describe the internal and unique technologies and characteristics of the solution that enable it to outperform Snowflake's offering both from a cost and performance perspective. We

In [0]:
dais_pdf["full_text"] = dais_pdf.apply(
    lambda row: f"""Title: {row["Title"]}
                Abstract:  {row["Abstract"]}""".strip(),
    axis=1,
)
print(dais_pdf.iloc[0]["full_text"])

Title: Nebula: The Journey of Scaling Instacart’s Data Pipelines with Apache Spark™ and Lakehouse
                Abstract:  Instacart has gone through immense growth during the pandemic and the trend continues. Instacart ads is no exception in this growth story. We have launched many new product lines including display and video ads covering the full advertising funnel to address the increasing demand of our retail partners. We have built advanced models to auto-suggest optimal bidding to increase the ROI for our CPG partners. Advertisers’ trust is the utmost priority and thus the quest to build a top-class ads measurement platform.
 
 Ads data processing requires complex data verifications to update ads serving stats. In ETL pipelines these were implemented through files containing thousands of lines of raw SQL which were hard to scale, test, and iterate upon. Our data engineers used to spend hours testing small changes due to a lack of local testing mechanisms. 
 
 These pain points

In [0]:
texts = []
for i, row in dais_pdf.iterrows():
    texts.append(row["full_text"])

## Question 1
Set up Chroma and create collection

In [0]:
import chromadb
from chromadb.config import Settings

chroma_client = chromadb.Client(
    Settings(
        chroma_db_impl="duckdb+parquet",
        persist_directory=DA.paths.user_db,  # this is an optional argument. If you don't supply this, the data will be ephemeral
    )
)

Using embedded DuckDB with persistence: data will be stored in: dbfs:/mnt/dbacademy-users/labuser4407984@vocareum.com/large-language-models/database.db


Fill out `collection_name` below.

In [0]:
# TODO
collection_name = "dais23_talks"

# If you have created the collection before, you need to delete the collection first
if len(chroma_client.list_collections()) > 0 and collection_name in [chroma_client.list_collections()[0].name]:
    chroma_client.delete_collection(name=collection_name)
else:
    print(f"Creating collection: '{collection_name}'")
    talks_collection = chroma_client.create_collection(name=collection_name)

No embedding_function provided, using default embedding function: SentenceTransformerEmbeddingFunction


Creating collection: 'dais23_talks'


In [0]:
# Test your answer. DO NOT MODIFY THIS CELL.

dbTestQuestion2_1(collection_name)

PASSED: All tests passed for lesson2, question1
RESULTS RECORDED: Click `Submit` when all questions are completed to log the results.


## Question 2

Add data to collection

In [0]:
# TODO
talks_collection.add(
    documents=dais_pdf["Title"][:100].tolist(),
    ids=[f"id{x}" for x in range(100)]
)

In [0]:
# Test your answer. DO NOT MODIFY THIS CELL.

dbTestQuestion2_2(talks_collection)

PASSED: All tests passed for lesson2, question2
RESULTS RECORDED: Click `Submit` when all questions are completed to log the results.


## Question 3

Query for relevant documents

In [0]:
# TODO
import json

results = talks_collection.query(
    query_texts=["space"],
    n_results=10
)

print(json.dumps(results, indent=4))

{
    "ids": [
        [
            "id53",
            "id57",
            "id10",
            "id17",
            "id86",
            "id88",
            "id56",
            "id38",
            "id32",
            "id64"
        ]
    ],
    "embeddings": null,
    "documents": [
        [
            "Bringing Data in-House! Behind the New York Jets' Winning Game Plan to Create a Data Lakehouse on Databricks",
            "Using Lakehouse to Fight Cancer: Ontada\u2019s Journey to Establish a RWD Platform on Databricks Lakehouse",
            "Simon + Denny Live: Ask Us Anything",
            "Journey Towards Uniting Metastores",
            "Clean Room Primer: Using Clean Rooms on Databricks to Utilize More and Better Data in your BI, ML, and Beyond",
            null,
            "Made in Italy: How Barilla Uses Databricks Lakehouse to Optimize its Operations",
            "Lakehouse as a FAIR Platform",
            "Multi-Cloud Data Governance on the Databricks Lakehouse",
      

In [0]:
# Test your answer. DO NOT MODIFY THIS CELL.

dbTestQuestion2_3(results)

PASSED: All tests passed for lesson2, question3
RESULTS RECORDED: Click `Submit` when all questions are completed to log the results.


## Question 4

Load language model

In [0]:
# TODO
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# Pick a model from HuggingFace that can generate text
model_id = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_id)
lm_model = AutoModelForCausalLM.from_pretrained(model_id)

pipe = pipeline(
    "text-generation", model=lm_model, tokenizer=tokenizer, max_new_tokens=512, device_map="auto", handle_long_generation="hole"
)

In [0]:
# Test your answer. DO NOT MODIFY THIS CELL.

dbTestQuestion2_4(pipe)

PASSED: All tests passed for lesson2, question4
RESULTS RECORDED: Click `Submit` when all questions are completed to log the results.


## Question 5

Prompt engineering for question answering

In [0]:
# TODO
# Come up with a question that you need the LLM assistant to help you with
# A sample question is "Help me find sessions related to XYZ"
question = "What's the latest news on space development?"

# Provide all returned similar documents from the cell above below
context = " ".join([f"#{str(i)}" for i in results["documents"][0]])

# Feel free to be creative how you construct the prompt. You can use the demo notebook as a jumpstart reference.
# You can also provide more requirements in the text how you want the answers to look like.
# Example requirement: "Recommend top-5 relevant sessions for me to attend."
prompt_template = f"Relevant context: {context}\n\n The user's question: {question}"

In [0]:
# Test your answer. DO NOT MODIFY THIS CELL.

dbTestQuestion2_5(question, context, prompt_template)

PASSED: All tests passed for lesson2, question5
RESULTS RECORDED: Click `Submit` when all questions are completed to log the results.


## Question 6 

Submit query for language model to generate response.

Hint: If you run into the error `index out of range in self`, make sure to check out this [documentation page](https://huggingface.co/docs/transformers/main_classes/pipelines#transformers.TextGenerationPipeline.__call__.handle_long_generation).

In [0]:
# TODO
lm_response = pipe(prompt_template)
print(lm_response[0]["generated_text"])

/databricks/python/lib/python3.10/site-packages/transformers/generation/utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Relevant context: #Bringing Data in-House! Behind the New York Jets' Winning Game Plan to Create a Data Lakehouse on Databricks #Using Lakehouse to Fight Cancer: Ontada’s Journey to Establish a RWD Platform on Databricks Lakehouse #Simon + Denny Live: Ask Us Anything #Journey Towards Uniting Metastores #Clean Room Primer: Using Clean Rooms on Databricks to Utilize More and Better Data in your BI, ML, and Beyond #None #Made in Italy: How Barilla Uses Databricks Lakehouse to Optimize its Operations #Lakehouse as a FAIR Platform #Multi-Cloud Data Governance on the Databricks Lakehouse #From Insights to Recommendations: How SkyWatch Predicts Demand for Satellite Imagery Using Databricks

 The user's question: What's the latest news on space development?

The user : No data for the next few years. Can a mobile app have any way to monitor these data flows, even if it already uses it? How is the data going to be used in the future? Who are the customers involved? How will the user interact wi

In [0]:
# Test your answer. DO NOT MODIFY THIS CELL.

dbTestQuestion2_6(lm_response)

PASSED: All tests passed for lesson2, question6
RESULTS RECORDED: Click `Submit` when all questions are completed to log the results.


Notice that the output isn't exactly helpful. Head on to using OpenAI to try out GPT-3.5 instead!

## OPTIONAL (Non-Graded): Use OpenAI models for Q/A

For this section to work, you need to generate an Open AI key. 

Steps:
1. You need to [create an account](https://platform.openai.com/signup) on OpenAI. 
2. Generate an OpenAI [API key here](https://platform.openai.com/account/api-keys). 

Note: OpenAI does not have a free option, but it gives you $5 as credit. Once you have exhausted your $5 credit, you will need to add your payment method. You will be [charged per token usage](https://openai.com/pricing). **IMPORTANT**: It's crucial that you keep your OpenAI API key to yourself. If others have access to your OpenAI key, they will be able to charge their usage to your account!

In [0]:
# TODO
import os

os.environ["OPENAI_API_KEY"] = "<FILL IN>"

In [0]:
import openai

openai.api_key = os.environ["OPENAI_API_KEY"]

If you would like to estimate how much it would cost to use OpenAI, you can use `tiktoken` library from OpenAI to get the number of tokens from your prompt.


We will be using `gpt-3.5-turbo` since it's the most economical option at ($0.002/1k tokens), as of May 2023. GPT-4 charges $0.04/1k tokens. The following code block below is referenced from OpenAI's documentation on ["Managing tokens"](https://platform.openai.com/docs/guides/chat/managing-tokens).

In [0]:
import tiktoken

price_token = 0.002
encoder = tiktoken.encoding_for_model("gpt-3.5-turbo")
cost_to_run = len(encoder.encode(prompt_template)) / 1000 * price_token
print(f"It would take roughly ${round(cost_to_run, 5)} to run this prompt")

We won't have to create a new vector database again. We can just send our `context` from above to OpenAI. We will use their chat completion API to interact with `GPT-3.5-turbo`. You can refer to their [documentation here](https://platform.openai.com/docs/guides/chat).

Something interesting is that OpenAI models use the system message to help their assistant to be more accurate. From OpenAI's [docs](https://platform.openai.com/docs/guides/chat/introduction):

> Future models will be trained to pay stronger attention to system messages. The system message helps set the behavior of the assistant.

In [0]:
# TODO
gpt35_response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": <FILL_IN>},
    ],
    temperature=0, # 0 makes outputs deterministic; The closer the value is to 1, the more random the outputs are for each time you re-run.
)

In [0]:
print(gpt35_response.choices[0]["message"]["content"])

In [0]:
from IPython.display import Markdown

Markdown(gpt35_response.choices[0]["message"]["content"])

We can also check how many tokens OpenAI has used

In [0]:
gpt35_response["usage"]["total_tokens"]

The results are noticeably much better compared to when using Hugging Face's GPT-2! It didn't get stuck in the text generation, but the sessions recommended are not all relevant to pandas either. You can further do more prompt engineering to get better results.

## Submit your Results (edX Verified Only)

To get credit for this lab, click the submit button in the top right to report the results. If you run into any issues, click `Run` -> `Clear state and run all`, and make sure all tests have passed before re-submitting. If you accidentally deleted any tests, take a look at the notebook's version history to recover them or reload the notebooks.

-sandbox
&copy; 2023 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="https://help.databricks.com/">Support</a>